# Meetup Feb 19

In [61]:
! [ ! -z "$COLAB_GPU" ] && pip install skorch

## Explore Data

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml

In [59]:
mnist_data = fetch_openml(data_id=554)

In [60]:
X = mnist_data['data'].astype('float32')
y = mnist_data['target'].astype('int64')

print(X.shape, y.shape)

(70000, 784) (70000,)


## Simple Neural Network Model

In [7]:
import torch.nn as nn

class SimpleFeedforward(nn.Module):
    def __init__(self):
        super().__init__()
        self.module = nn.Sequential(
            nn.Linear(784, 98),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(98,10)
        )
    def forward(self, X):
        return self.module(X)

In [8]:
from sklearn.model_selection import train_test_split

X_scaled = X / X.max()

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42)

In [9]:
from skorch import NeuralNet

net = NeuralNet(
    SimpleFeedforward,
    criterion=nn.CrossEntropyLoss,
    max_epochs=10,
    lr=0.3,
    # device='cuda', # uncomment out to run on gpu
)

In [10]:
_ = net.fit(X_train, y_train)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.5775        0.2973  1.1797
      2        0.3208        0.2521  1.4316
      3        0.2710        0.1968  1.2767
      4        0.2397        0.1940  1.1888
      5        0.2192        0.1624  1.2892
      6        0.2069        0.1385  1.1394
      7        0.1936        0.1300  1.1347
      8        0.1856        0.1268  1.1136
      9        0.1784        0.1310  1.1285
     10        0.1715        0.1239  1.0846


In [11]:
net.set_params(max_epochs=5)
_ = net.partial_fit(X_train, y_train)

     11        0.1609        0.1173  1.0857
     12        0.1588        0.1164  1.1218
     13        0.1551        0.1202  1.0993
     14        0.1520        0.1080  1.3218
     15        0.1540        0.1319  1.3325


## History

In [12]:
len(net.history)

15

In [13]:
net.history[-1, 'valid_loss']

0.13188647115798224

In [14]:
net.history[-3:, 'train_loss']

[0.1551400553612482, 0.1520235111486344, 0.15403895963941303]

## Callbacks

### EpochScoring

In [54]:
from sklearn.metrics import make_scorer
from skorch.callbacks import EpochScoring

def accuracy_argmax(y_true, y_pred):
    return np.mean(y_true == np.argmax(y_pred, axis=-1))
accuracy_argmax_scorer = make_scorer(accuracy_argmax)

epoch_acc = EpochScoring(
    accuracy_argmax_scorer,
    name='valid_acc',
    lower_is_better=False)

## Checkpointing

In [19]:
from skorch.callbacks import Checkpoint

In [21]:
cp = Checkpoint(monitor='valid_acc_best', dirname='exp_01')

In [46]:
net = NeuralNet(
    SimpleFeedforward,
    criterion=nn.CrossEntropyLoss,
    max_epochs=15,
    lr=0.8,
    # device='cuda', # uncomment out to run on gpu
    callbacks=[epoch_acc, cp]
)

In [47]:
_ = net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.5531       0.8747        0.4470     +  0.7692
      2        0.3142       0.9126        0.2912     +  0.7537
      3        0.2641       0.9539        0.1579     +  0.7501
      4        0.2427       0.9488        0.1874        0.7449
      5        0.2268       0.9554        0.1484     +  0.7306
      6        0.2141       0.9543        0.1516        0.7583
      7        0.2058       0.9529        0.1734        0.7170
      8        0.1987       0.9533        0.1547        0.7559
      9        0.1918       0.9591        0.1440     +  0.7329
     10        0.1818       0.9586        0.1391        0.7389
     11        0.1789       0.9656        0.1190     +  0.7413
     12        0.1687       0.9527        0.1630        0.7221
     13        0.1697       0.9623        0.1210        0.7429
     14        0.1646       0.9642        0.1196       

In [55]:
net.load_params(checkpoint=cp)
y_pred = net.predict(X_test)
print('test accuracy:', accuracy_argmax(y_test, y_pred))

test accuracy: 0.9629714285714286


## Integration with scikit-learn

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [51]:
pipe = Pipeline([
    ('min_max', MinMaxScaler()),
    ('net', net)
])

In [52]:
_ = pipe.fit(X_train, y_train)

Re-initializing module because the following parameters were re-set: .
Re-initializing optimizer because the following parameters were re-set: .
  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.5913       0.8948        0.3367     +  0.8206
      2        0.3410       0.9182        0.2595     +  0.8401
      3        0.2980       0.9440        0.1805     +  0.8322
      4        0.2656       0.9449        0.1878     +  0.8420
      5        0.2534       0.9557        0.1393     +  0.8353
      6        0.2342       0.9303        0.2460        0.8427
      7        0.2293       0.9574        0.1389     +  0.8101
      8        0.2215       0.9465        0.1661        0.7980
      9        0.2162       0.9532        0.1593        0.8075
     10        0.2123       0.9556        0.1474        0.7320
     11        0.2021       0.9618        0.1304     +  0.7400
     12        0.2014       0.9664  